In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.qa.jerxrm import answer_question, answer_question_with_reasoning, answer_question_with_triplets, answer_question_reasoning_with_triplets
from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark

set_seed(89)

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)

In [ ]:
# from datetime import datetime
# import random
# random.seed(datetime.now())
# ABLATION_RECORD_IDS = random.sample(df['id'].tolist(), 100)
# ABLATION_RECORD_IDS


In [ ]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
qd_df = pd.read_json('../../data/generated/musique-evaluation/question-decomposition.jsonl', orient='records', lines=True)
df = pd.merge(df.drop(columns=['question', 'question_decomposition']), qd_df, on='id', suffixes=('', ''))
print(df.shape)
df.head()

(100, 8)


,id,paragraphs,answer,answer_aliases,answerable,answers,question,question_decomposition
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar],Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Who is Mahmoud Mi..."
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",Berrien County,[Berrien County],True,[Berrien County],In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'To which city is ..."
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River],For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'On which river is..."
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith],Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Where was Tebesa ..."
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Snapper Foster,[Snapper Foster],True,[Snapper Foster],Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Who performed Nig..."


In [ ]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [ ]:
results = []

In [ ]:
_, scores = benchmark(df, answer_question, perfect_retrieval_func, ignore_errors=True)
results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "direct"})
jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.56,
  "f1": 0.6546596279537455,
  "fuzzy_match": 0.68
}


In [ ]:
_, scores = benchmark(df, answer_question_with_reasoning, perfect_retrieval_func, ignore_errors=True)
results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "reasoning"})
jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.52,
  "f1": 0.6476129426129426,
  "fuzzy_match": 0.66
}


In [ ]:
_, scores = benchmark(df, answer_question_with_triplets, perfect_retrieval_func, ignore_errors=True)
results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "triplets"})
jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.6,
  "f1": 0.6785079365079365,
  "fuzzy_match": 0.66
}


In [ ]:
_, scores = benchmark(df, answer_question_reasoning_with_triplets, perfect_retrieval_func, ignore_errors=True)
results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "reasoning+triplets"})
jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.57,
  "f1": 0.6983598362421891,
  "fuzzy_match": 0.71
}


# Report

In [ ]:
report_df = pd.DataFrame.from_records(results, columns=['context', 'retrieval', 'qa', 'exact_match', 'fuzzy_match', 'f1'])
report_df.drop(columns=['fuzzy_match'], inplace=True)
report_df

,context,retrieval,qa,exact_match,f1
0,paragraphs,groundtruth,direct,0.56,0.654660
1,paragraphs,groundtruth,reasoning,0.52,0.647613
2,paragraphs,groundtruth,triplets,0.60,0.678508
3,paragraphs,groundtruth,reasoning+triplets,0.57,0.698360
